In [ ]:
import os
import numpy as np
from PIL import Image

base_dir = "."  # adjust if needed
input_img_path = os.path.join(base_dir, "sar_1_gray.jpg")

# 1. Load image and convert to grayscale (monochrome)
img = Image.open(input_img_path).convert("L")
img_array = np.array(img)
print("Image shape:", img_array.shape, "dtype:", img_array.dtype)

# 1a. Save monochrome image as raw per-pixel binary file
raw_path = os.path.join(base_dir, "sar_1_gray_raw.bin")
img_array.tofile(raw_path)
raw_size = os.path.getsize(raw_path)
print("Raw per-pixel file size (bytes):", raw_size)


In [ ]:
def haar2d(arr: np.ndarray):
    """One-level 2D Haar transform for a 2D grayscale image."""
    h, w = arr.shape
    assert h % 2 == 0 and w % 2 == 0, "Image dimensions must be even for one-level Haar transform"
    arr = arr.astype(float)
    
    # Row-wise transform
    temp = np.zeros_like(arr, dtype=float)
    for i in range(h):
        row = arr[i, :]
        a = (row[0::2] + row[1::2]) / 2.0
        d = (row[0::2] - row[1::2]) / 2.0
        temp[i, :w//2] = a
        temp[i, w//2:] = d
    
    # Column-wise transform
    coeffs = np.zeros_like(temp, dtype=float)
    for j in range(w):
        col = temp[:, j]
        a = (col[0::2] + col[1::2]) / 2.0
        d = (col[0::2] - col[1::2]) / 2.0
        coeffs[:h//2, j] = a
        coeffs[h//2:, j] = d
    
    LL = coeffs[:h//2, :w//2]
    LH = coeffs[:h//2, w//2:]
    HL = coeffs[h//2:, :w//2]
    HH = coeffs[h//2:, w//2:]
    return LL, LH, HL, HH

LL, LH, HL, HH = haar2d(img_array)
print("LL shape:", LL.shape)
print("LH shape:", LH.shape)
print("HL shape:", HL.shape)
print("HH shape:", HH.shape)


In [ ]:
def quantize_hf(LH, HL, HH, num_levels=4):
    """Uniform scalar quantization of high-frequency components to num_levels."""
    hf = np.concatenate([LH.flatten(), HL.flatten(), HH.flatten()])
    min_v = hf.min()
    max_v = hf.max()
    if min_v == max_v:
        return (np.zeros_like(LH), np.zeros_like(HL), np.zeros_like(HH)), (min_v, max_v)
    
    bins = np.linspace(min_v, max_v, num_levels + 1)
    levels = 0.5 * (bins[:-1] + bins[1:])
    
    def quantize_block(block):
        flat = block.flatten()
        idx = np.digitize(flat, bins[1:-1], right=False)
        q_flat = levels[idx]
        return q_flat.reshape(block.shape)
    
    qLH = quantize_block(LH)
    qHL = quantize_block(HL)
    qHH = quantize_block(HH)
    return (qLH, qHL, qHH), (min_v, max_v)

(qLH, qHL, qHH), hf_range = quantize_hf(LH, HL, HH, num_levels=4)
print("High-frequency range:", hf_range)


In [ ]:
def rle_encode(arr: np.ndarray):
    """Run-length encode a 2D array, return values and counts."""
    flat = arr.flatten()
    if flat.size == 0:
        return np.array([]), np.array([], dtype=int)
    
    values = [flat[0]]
    counts = [1]
    for x in flat[1:]:
        if x == values[-1]:
            counts[-1] += 1
        else:
            values.append(x)
            counts.append(1)
    return np.array(values), np.array(counts, dtype=int)

LH_vals, LH_counts = rle_encode(qLH)
HL_vals, HL_counts = rle_encode(qHL)
HH_vals, HH_counts = rle_encode(qHH)

print("LH RLE length:", len(LH_vals))
print("HL RLE length:", len(HL_vals))
print("HH RLE length:", len(HH_vals))


In [ ]:
# Save transformed data in binary form in the order LL, LH, HL, HH.
# LL is stored as a full matrix, LH/HL/HH as (value, count) pairs (RLE).

encoded_path = os.path.join(base_dir, "haar_haar_rle_encoded.npz")
np.savez_compressed(
    encoded_path,
    LL=LL.astype(np.float32),
    LH_vals=LH_vals.astype(np.float32),
    LH_counts=LH_counts.astype(np.int32),
    HL_vals=HL_vals.astype(np.float32),
    HL_counts=HL_counts.astype(np.int32),
    HH_vals=HH_vals.astype(np.float32),
    HH_counts=HH_counts.astype(np.int32),
    shape=np.array(img_array.shape, dtype=np.int32),
)

encoded_size = os.path.getsize(encoded_path)
print("Encoded file size (bytes):", encoded_size)
print("Compression ratio (encoded/raw):", encoded_size / raw_size)
